In [1]:
# import the general stuff
import os
from os import getcwd
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from sklearn import metrics

#import stuff for gradCAM
import matplotlib.cm as cm
from IPython.display import Image

#Set GPU to use
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

#import the tf stuff
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model

from tensorflow.keras.preprocessing.image import ImageDataGenerator

#Check TensorFlow Version
print('TF version: {}' .format(tf.__version__))

#Check for GPU utilization
if tf.test.gpu_device_name():
    print(' GPU: {}'.format(tf.test.gpu_device_name()))
else:
    print("No GPU")

TF version: 2.3.2
 GPU: /device:GPU:0


In [2]:
#set Image size (RGB so imshape is 3)
pix_dim = 512
imsize = (pix_dim,pix_dim) 
imshape = (pix_dim,pix_dim,3)

#load model
filepath = './models/VGG16_model_4_8020split_512px'
model = tf.keras.models.load_model(filepath, compile = True)
model.summary()

Model: "functional_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 512, 512, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 512, 512, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 512, 512, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 256, 256, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 256, 256, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 256, 256, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, 128, 128)    

In [3]:
last_conv_layer_name = "block5_pool"
classifier_layer_names = [
    "flatten_1",
    "dense_2",
    "dropout_1",
    "dense_3",
]

In [4]:
#make a list of images names

def get_image_list(hxpath):
    labels = []
    for img in os.listdir(hxpath):
        #pull the name of the image
        labels.append(img)
    return labels

In [5]:
def get_img_array(img_path, size):
    # `img` is a PIL image 
    img = tf.keras.preprocessing.image.load_img(img_path, target_size = imsize)
    # `array` is a float32 Numpy array 
    array = tf.keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, X, X, 3)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(
    img_array, model, last_conv_layer_name, classifier_layer_names
):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer
    last_conv_layer = model.get_layer(last_conv_layer_name)
    last_conv_layer_model = Model(model.inputs, last_conv_layer.output)

    # Second, we create a model that maps the activations of the last conv
    # layer to the final class predictions
    classifier_input = tf.keras.Input(shape=last_conv_layer.output.shape[1:])
    x = classifier_input
    for layer_name in classifier_layer_names:
        x = model.get_layer(layer_name)(x)
    classifier_model = tf.keras.Model(classifier_input, x)

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        # Compute activations of the last conv layer and make the tape watch it
        last_conv_layer_output = last_conv_layer_model(img_array)
        tape.watch(last_conv_layer_output)
        # Compute class predictions
        preds = classifier_model(last_conv_layer_output)
        top_pred_index = tf.argmax(preds[0])
        top_class_channel = preds[:, top_pred_index]
        

    # This is the gradient of the top predicted class with regard to
    # the output feature map of the last conv layer
    grads = tape.gradient(top_class_channel, last_conv_layer_output)
    
    #grads = tape.gradient(bottom_class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    last_conv_layer_output = last_conv_layer_output.numpy()[0]
    pooled_grads = pooled_grads.numpy()
    for i in range(pooled_grads.shape[-1]):
        last_conv_layer_output[:, :, i] *= pooled_grads[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(last_conv_layer_output, axis=-1)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
    return heatmap

In [7]:
# Prepare image
def make_gradcam_images(hxpath, GCfolderpath):
    
    #get the list of labels
    labels = get_image_list(hxpath)

    for ip in labels: 
        impath = str(hxpath) + str(ip)
        img = tf.keras.preprocessing.image.load_img(impath,target_size = imsize)
        img = tf.keras.preprocessing.image.img_to_array(img)
        img = img/255
        img_array = np.expand_dims(img,axis=0)


        # Generate class activation heatmap
        heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name, classifier_layer_names)

        # We load the original image
        Orimg = tf.keras.preprocessing.image.load_img(impath, target_size = imsize)
        Orimg = tf.keras.preprocessing.image.img_to_array(Orimg)

        # We rescale heatmap to a range 0-255
        heatmap = np.uint8(255 * heatmap)

        # We use jet colormap to colorize heatmap
        vir = cm.get_cmap("viridis")

        # We use RGB values of the colormap
        vir_colors = vir(np.arange(256))[:, :3]
        vir_heatmap = vir_colors[heatmap]

        # We create an image with RGB colorized heatmap
        vir_heatmap = tf.keras.preprocessing.image.array_to_img(vir_heatmap)
        vir_heatmapI = vir_heatmap.resize((Orimg.shape[1], Orimg.shape[0]))
        vir_heatmap = tf.keras.preprocessing.image.img_to_array(vir_heatmapI)

        # Superimpose the heatmap on original image
        superimposed_img = vir_heatmap * 0.4 + Orimg
        superimposed_img = tf.keras.preprocessing.image.array_to_img(superimposed_img)

        #save
        superimposed_img = superimposed_img.save(GCfolderpath + ip) 

In [8]:
#CALL THE FUNCTION AND MAKE THE IMAGES
make_gradcam_images('../data/pics/raw/Isaias_20200804a_jpgs/','../data/pics/gradcam/Model4/I_GCimages/')
make_gradcam_images('../data/pics/raw/Florence_20180917a_jpgs/','../data/pics/gradcam/Model4/F_GCimages/')
make_gradcam_images('../data/pics/raw/Michael_20181011a_jpgs/','../data/pics/gradcam/Model4/M_GCimages/')


<ipython-input-5-abc1a3fd6958>:61: RuntimeWarning: invalid value encountered in true_divide
  heatmap = np.maximum(heatmap, 0) / np.max(heatmap)
